### 1. Multiple Regression

Consider energy efficiency data  UCI repository which is supplied by Athanasios Tsanas and Angeliki Xifara. It contains 10 attributes :
1. X1 - Relative Compactness : continuous data with 0.62 to 0.98
2. X2 - Surface Area : continuous data with 514.50 to 808.50
3. X3 - Wall Area : continuous data with 245.00 to 416.50
4. X4 - Roof Area : continuous data with 110.25 to 220.50
5. X5 - Overall Height : continuous data with 3.50 to 7.00
6. X6 - Orientation : integer data with 2.00 to 5.00
7. X7 - Glazing Area : continuous data with 0.00 to 0.40
8. X8 - Glazing Area Distribution : integer data with 0.00 to 5.00
9. Y1 - Heating Load : continuous data with 6.01 to 43.10
10. Y2 - Cooling Load : continuous data with 10.90 to 48.03


The target is the $9^{th}$ attribute, the response variable Y1 and do not use Y2 in the model (but if you want to use, you can try).

There is no missing values.

You must implement the multiple regression using **SGD** from our class with average square loss.
- The total 10 points will give to a student who can find the parameters of this regression and achieve 0.01 for the average square loss.
- The total 7 points will give to a student who can find the parameters of this regression and achieve 0.1 for the average square loss.
- The total 5 points will give to a student who can make this code work with this dataset.

This data addresses in https://archive.ics.uci.edu/static/public/242/energy+efficiency.zip . Hint : you have to use !wget and !unzip this data from !wget and you can use pandas library and sklearn.preprocessing  to deal with data. TA's average square loss from last epoch is around 0.006. I think you can do better than me. Cheer up!! you can do this.


In [1]:
# Must run this statement first
!pip install --quiet mxnet

In [9]:
# We will need to process this text dataset using pandas
# Deal with data in this block
import mxnet as mx
from mxnet import nd
import pandas as pd
import numpy as np

# !wget https://archive.ics.uci.edu/static/public/242/energy+efficiency.zip
# !unzip ./energy+efficiency.zip

df = pd.read_excel('ENB2012_data.xlsx')

df.drop(["Y2"], axis=1, inplace=True)

# df.head(10)

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# df.describe()

min_max_scaler = MinMaxScaler()
df_normalized = pd.DataFrame(min_max_scaler.fit_transform(df), columns=df.columns)

target = nd.array(df_normalized["Y1"])
features = nd.array(df_normalized.drop("Y1", axis=1))

# print(target.shape, features.shape)



In [10]:
# Build multiple regression from ground-up
import mxnet as mx
from mxnet import nd, autograd, gluon

# Hyper parameter
batch_size = 10
lr = 0.005
epochs = 20

trainset = gluon.data.ArrayDataset(features, target)
trainloader = gluon.data.DataLoader(trainset, batch_size, shuffle=True)

from mxnet.gluon import nn
from mxnet import init

model = nn.Sequential()
model.add(nn.Dense(1))
model.initialize(init.Normal(sigma=0.01))

criterion = gluon.loss.L2Loss()
optimizer = gluon.Trainer(model.collect_params(), 'sgd', {'learning_rate': lr})

# Traning Loop
for epoch in range(epochs):
  for features, target in trainloader:
    with autograd.record():
      logits = model(features)
      loss = criterion(logits, target)
    loss.backward()
    optimizer.step(batch_size)
  loss = criterion(model(features), target)
  print(f'epoch {epoch + 1}, loss: {loss.mean().asnumpy()}')


# Model parameters
weight = model[0].weight.data().asnumpy()
bias = model[0].bias.data().asnumpy()

print(f"Model's Weight:\n {weight}")
print(f"Model's Bias:\n {bias}")











epoch 1, loss: [0.01802309]
epoch 2, loss: [0.04259727]
epoch 3, loss: [0.02673271]
epoch 4, loss: [0.00929978]
epoch 5, loss: [0.02260777]
epoch 6, loss: [0.01039464]
epoch 7, loss: [0.00400636]
epoch 8, loss: [0.00856073]
epoch 9, loss: [0.00666468]
epoch 10, loss: [0.00874935]
epoch 11, loss: [0.0044263]
epoch 12, loss: [0.00520474]
epoch 13, loss: [0.00377015]
epoch 14, loss: [0.01466225]
epoch 15, loss: [0.00347824]
epoch 16, loss: [0.00300786]
epoch 17, loss: [0.00235405]
epoch 18, loss: [0.01052281]
epoch 19, loss: [0.00951543]
epoch 20, loss: [0.00189464]
Model's Weight:
 [[ 1.13266625e-01  1.10821944e-04  1.61712468e-01 -1.04915150e-01
   2.98812062e-01  1.50882713e-02  1.46970242e-01  5.92767149e-02]]
Model's Bias:
 [0.11378725]
